In [2]:
import os

In [3]:
%pwd

'c:\\Users\\autom\\OneDrive\\Desktop\\End-to-end-Machine-Learning-Project-with-MLflow-Salary\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\autom\\OneDrive\\Desktop\\End-to-end-Machine-Learning-Project-with-MLflow-Salary'

In [21]:
import pandas as pd

In [22]:
data = pd.read_csv("artifacts/data_ingestion/Salary.csv")
data.head()

,Index,YearsExperience,Salary
0,0,1.2,39344.0
1,1,1.4,46206.0
2,2,1.6,37732.0
3,3,2.1,43526.0
4,4,2.3,39892.0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            30 non-null     int64  
 1   YearsExperience  30 non-null     float64
 2   Salary           30 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 848.0 bytes


In [24]:
data.isnull().sum()

Index              0
YearsExperience    0
Salary             0
dtype: int64

In [25]:
data.shape

(30, 3)

In [26]:
data.columns

Index(['Index', 'YearsExperience', 'Salary'], dtype='object')

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [28]:
from MlflowProject.constants import *
from MlflowProject.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [30]:
import os
from MlflowProject import logger

In [31]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [33]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-05-26 12:20:16,590: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-26 12:20:16,594: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-26 12:20:16,598: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-26 12:20:16,600: INFO: common: created directory at: artifacts]
[2025-05-26 12:20:16,604: INFO: common: created directory at: artifacts/data_validation]
